This is a practice of using ED method to solve 1D $S=\frac{1}{2}$ Heisenberg Model. Follow the instruction of arXiv: 1102.4006v1 and the ED example of Wei Zheng.

\section{Hamiltonian}
\begin{equation}
    H = \sum_{i=1, \alpha}^{L-1} J_\alpha S_i^\alpha  S_{i+1}^\alpha
\end{equation}

where $L$ is the total number of spins, $\alpha = x,y,z$.

Here we use the XXZ model, in which $J_x = J_y = J_{xy}$:
\begin{align}
    H_{XXZ} & = \sum_{i=1}^{L-1} [J_{xy} (S_i^x  S_{i+1}^x + S_i^y  S_{i+1}^y) + J_z S_i^z  S_{i+1}^z]\\
            & = \sum_{i=1}^{L-1} [\frac{J_{xy}}{2} (S_i^+  S_{i+1}^- + S_{i+1}^+ S_i^-  ) + J_z S_i^z  S_{i+1}^z]
\end{align}

where $S^\pm = S^x \pm i S^y$.

\section{Basis generation}

We use the eigen states of course we $S^z$ as basis states and use $|0 \rangle$ to represent $|\downarrow \rangle$ and $|1 \rangle$ to represent $|\uparrow \rangle$. The states can thus be represented by a set of binary numbers. Moreover, we can use the corresponding dicimal number of each state as its tag.

For a system with N sites, the dimension of Hilbert space is $2^N$. The Hilbert space dimension can be furtuer reduced by applying symmetries and conservation laws.

For fermions and hardcore bosons, the occupation number of each site can only be $0$ or $1$, the basis generation always seems simple and straightforward. However, when comes to bosons, we will need a little trick for the basis generation. Here, we only take the easy way and a more general and complicate method will be introduced in the note of solving Bose Hubbard model.

For tutorial purposes, we give the example in both non-conserved $S_z$ and conserved $S_z$ case. 

\subsection{Non-conserved $S_z$}

For non-conserved $S_z$, we use the eigen states of course we $S^z$ as basis states and use $|0 \rangle$ to represent $|\downarrow \rangle$ and $|1 \rangle$ to represent $|\uparrow \rangle$. The states can thus be represented by a set of binary numbers. Moreover, we can use the corresponding dicimal number of each state as its tag.

In [216]:
import numpy as np
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import eigsh

def StateConfig(SiteNum, tag):
    '''The configuration of the state with tag.
    
       Setups required: 
           dim: Hilbert space dimension
           SiteNum: total site number
           
       Return: binary form of tag. type: list'''
    if tag >= dim:
        return print('Error: The tag is out of range.')
    else:
        b = bin(tag)[2:]
        b = b.rjust(SiteNum,'0') 
        return b
    
def flip(SiteNum, tag, i, j):
    '''Flip the spin on i,j site.
    
       Setups required: 
           SiteNum: total site number
           
       Return: The tag of new state, type: int '''
    f = pow(2, SiteNum - i -1) + pow(2, SiteNum - j -1)
    Newtag = tag^f
    return Newtag

def heisenberg(SiteNum, Jz, Jxy, periodic = False):
    '''CSC formation of XXZ model Hamiltonian
       Inputs: SiteNum : the site number
               
       Outputs: positions and values of non-zero Hamiltonian elements'''
    
    dim = pow(2,SiteNum)

    if periodic:
        limit = SiteNum
    else:
        limit = SiteNum - 1

    col = []
    row = []
    data = []

    for tag in range(dim):
        si = StateConfig(SiteNum, tag)
        temp = {}
        for i in range(limit):
            next = (i+1) % SiteNum
            
            '''Hz'''
            if tag in temp:
                temp[tag] += Jz * (int(si[i]) - 1/2) * (int(si[next]) - 1/2)
            else:
                temp[tag] = Jz * (int(si[i]) - 1/2) * (int(si[next]) - 1/2)
            
            '''Hxy'''
            if si[i] != si[next]:
                ntag = flip(SiteNum, tag, i, next)
                if ntag in temp:
                    temp[ntag] += Jxy / 2
                else:
                    temp[ntag] = Jxy / 2
                    
        odTemp = sorted(temp)
        for sf in odTemp:
            col.append (tag)
            row.append (sf)
            data.append (temp[sf])

    return col, row, data
SiteNum = 10
dim = pow(2, SiteNum)

Jz = 1
Jxy = 1

col, row, data = heisenberg(SiteNum, Jz, Jxy, periodic = True)
ham = csc_matrix ((data, (row, col)), shape=(dim, dim))

vals, vecs = eigsh (ham, k=5)
print(vals)

[-4.51544635 -4.09220735 -4.09220735 -4.09220735 -3.77059744]


\subsection{Conserved $S_z$}

For conserved $S_z = \frac{1}{2} (n_\uparrow - n_\downarrow)$, the number of states are $dim = \frac{N!}{n_\uparrow ! n_\downarrow !}$. We just loop over all the possible states and check whether $n_\uparrow$ is equal to the target sector. 

In [14]:
import numpy as np
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import eigsh

'''Selected states'''
def StateConfig(SiteNum, tag):
    '''The configuration of the state with tag.
    
       Setups required: 
           dim: Hilbert space dimension
           SiteNum: total site number
           
       Return: binary form of tag. type: list'''
    total = pow(2, SiteNum)
    if tag >= total:
        return print('Error: The tag is out of range.')
    else:
        b = bin(tag)[2:]
        b = b.rjust(SiteNum,'0') 
        return b
    
def mz(tag, SiteNum):
    '''mz of a given state, which is labelled by a integer tag
       Inputs: tag: tag of the input state
               SiteNum : total number of spins
       Outputs: mz'''
    b = StateConfig(SiteNum, tag)
    mz = 0
    for i in range(SiteNum):
        if b[i] == '0':
            mz -= 0.5
        else:
            mz += 0.5
    return mz

def BasisList(Sz, SiteNum):
    '''Basis list of a spin chain with conserved Sz
       Inputs: Sz: required Sz
               SiteNum: total number of spins
       Outpus: basis: basis list'''
    basis = []
    total = pow(2, SiteNum)
    for i in range(total):
        if mz(i,SiteNum) == Sz:
            basis.append(i)
        else:
            continue
    return basis

def flip(SiteNum, tag, i, j):
    '''Flip the spin on i,j site.
    
       Setups required: 
           SiteNum: total site number
           
       Return: The tag of new state, type: int '''
    f = pow(2, SiteNum - i -1) + pow(2, SiteNum - j -1)
    Newtag = tag^f
    return Newtag

def heisenberg_FixSz(SiteNum, Jz, Jxy, Sz, periodic = False):
    '''CSC formation of XXZ model Hamiltonian with fixed Sz
       Inputs: SiteNum : the site number    
       Outputs: positions and values of non-zero Hamiltonian elements'''
    basis = BasisList(Sz, SiteNum)
    dim = len(basis)

    if periodic:
        limit = SiteNum
    else:
        limit = SiteNum - 1

    col = []
    row = []
    data = []

    for tag in basis:
        si = StateConfig(SiteNum, tag)
        ind = basis.index(tag)
        temp = {}
        for i in range(limit):
            next = (i+1) % SiteNum
            
            '''Hz'''
            if ind in temp:
                temp[ind] += Jz * (int(si[i]) - 1/2) * (int(si[next]) - 1/2)
            else:
                temp[ind] = Jz * (int(si[i]) - 1/2) * (int(si[next]) - 1/2)
            
            '''Hxy'''
            if si[i] != si[next]:
                ntag = flip(SiteNum, tag, i, next)
                nind = basis.index(ntag)
                if nind in temp:
                    temp[nind] += Jxy / 2
                else:
                    temp[nind] = Jxy / 2
                    
        odTemp = sorted(temp)
        for sf in odTemp:
            col.append (ind)
            row.append (sf)
            data.append (temp[sf])

    return col, row, data


In [15]:
def fac(N):
    return np.math.factorial(N)

SiteNum = 16
Sz = 0
dim = fac(SiteNum)/(fac(SiteNum/2 + Sz) * fac(SiteNum/2 - Sz))
print('Hilbert space dimension is:', dim)
dim = int(dim)

Jz = 1
Jxy = 1

col, row, data = heisenberg_FixSz(SiteNum, Jz, Jxy, Sz, periodic = True)
ham = csc_matrix ((data, (row, col)), shape=(dim, dim))

vals, vecs = eigsh (ham, k=5, which = 'SA')
print(vals)

Hilbert space dimension is: 12870.0
[-7.14229636 -6.87210668 -6.69654743 -6.52340706 -6.52340706]


\section{Apply Other Symmetries}
\subsection{Translation symmetry: Momentum States}

For a system with N sites, $Sz = -\frac{N}{2}, -\frac{N}{2}+1, ..., \frac{N}{2}$. If the system has translation symmetry, the allowed momentums are $k = m \frac{2\pi}{N}, m = -\frac{N}{2}+1, -\frac{N}{2}+2, ..., \frac{N}{2}$ for $N = even$ and $k = m \frac{2\pi}{N}, m = -N//2, -N//2+1, ..., N//2$ for $N = odd$. Here we set $N = even$ for simplicity.

Here we use $\frac{2\pi}{N}$ as the unit of the momentum.

In [2]:
import numpy as np
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import eigsh

'''Momentum states generation'''
def StateConfig(SiteNum, tag):
    '''The configuration of the state with tag.
    
       Setups required: 
           dim: Hilbert space dimension
           SiteNum: total site number
           
       Return: binary form of tag. type: list'''
    total = pow(2, SiteNum)
    if tag >= total:
        return print('Error: The tag is out of range.')
    else:
        b = bin(tag)[2:]
        b = b.rjust(SiteNum,'0') 
        return b

def mz(tag, SiteNum):
    '''mz of a given state, which is labelled by a integer tag
       Inputs: tag: tag of the input state
               SiteNum : total number of spins
       Outputs: mz'''
    b = StateConfig(SiteNum, tag)
    mz = 0
    for i in range(SiteNum):
        if b[i] == '0':
            mz -= 0.5
        else:
            mz += 0.5
    return mz

def BasisList(Sz, SiteNum):
    '''Basis list of a spin chain with conserved Sz
       Inputs: Sz: required Sz
               SiteNum: total number of spins
       Outpus: basis: basis list'''
    basis = []
    total = pow(2, SiteNum)
    for i in range(total):
        if mz(i,SiteNum) == Sz:
            basis.append(i)
        else:
            continue
    return basis

def cyclebits(n, tag, SiteNum):
    '''Performs a cyclic permutations to the right of the n first bits 
       of the integer tag.
       Inputs: tag: tag of the reference state
               n: cycle times
               SiteNum: total number of sites
       Outputs: ntag: the tag of the outcoming state'''
    b = StateConfig(SiteNum, tag)
    b1 = b[:n]
    b2 = b[n:]
    nb = b2 + b1
    nb = [str(x) for x in nb]
    nb_str = "".join(nb)
    ntag = int(nb_str, 2)    
    return ntag

def MomBasis(k, Sz, SiteNum):
    '''Momentum basis list of a given k in Sz sector
       Inputs: k: momentum 
               SiteNum: total number of spins
       Outpus: mbasis: momentum basis list
               Ra: list of corresponding cycle period'''
    mbasis = []
    Ra = []
    basis = BasisList(Sz, SiteNum)
    
    while basis !=[]:
        tag = basis[0]
        stag = basis[0]
        for n in range(1,SiteNum + 1):
            ntag = cyclebits(n, tag, SiteNum)
            if ntag < tag:
                basis.remove(ntag)
                if ntag < stag:
                    stag = ntag
            elif ntag > tag:
                basis.remove(ntag)
            elif ntag == tag:
                if k%(SiteNum/n) == 0:
                    Ra.append(n)
                    mbasis.append(stag)
                basis.remove(ntag)
                break
    return mbasis, Ra

In [1]:
'''Hamiltonian generation'''
def findstate(tag, mbasis, SiteNum):
    '''Find the reference state of an arbitrary state.
       Inputs: tag :tag of the input state
               mbasis: list of momentum basis
               SiteNum: total number of sites
       Outputs: ntag: tag of the corresponding reference state
                l : translation steps
                ntag and l will return negative values if tag is not in mbasis.'''
    l = -1
    count = 0
    b = StateConfig(SiteNum, tag)
    for n in range(SiteNum + 1):
        count +=1
        ntag = cyclebits(n, tag, SiteNum)
        if ntag in mbasis:
            l = n
            break
        else:
            pass
    if count > SiteNum:
        ntag = -1
    return ntag, l

def flip(SiteNum, tag, i, j):
    '''Flip the spin on i,j site.
    
       Setups required: 
           SiteNum: total site number
           
       Return: The tag of new state, type: int '''
    f = pow(2, SiteNum - i -1) + pow(2, SiteNum - j -1)
    Newtag = tag^f
    return Newtag

def heisenberg_T(SiteNum, Jz, Jxy, Sz = 0, k = 0):
    '''CSC formation of XXZ model Hamiltonian with translation symmetry
       Inputs: SiteNum : the site number
               Sz: total Sz sector
               k : given momentum
       Outputs: positions and values of non-zero Hamiltonian elements
                dim : dimension of Hilbert space'''
    fac = -1j * 2 * np.pi / SiteNum
    
    mb, R = MomBasis(k, Sz, SiteNum)
    dim = len(mb)
    
    col = []
    row = []
    data = []
    
    '''Hamiltonian generation'''
    for tag in mb:
        temp = {}
        ind  = mb.index(tag)
        si = StateConfig(SiteNum, tag)
        for i in range(SiteNum):
            next = (i+1) % SiteNum
            
            '''Hz'''
            if ind in temp:
                temp[ind] += Jz * (int(si[i]) - 1/2) * (int(si[next]) - 1/2)
            else:
                temp[ind] = Jz * (int(si[i]) - 1/2) * (int(si[next]) - 1/2)
            
            '''Hxy'''
            if si[i] != si[next]:
                ntag = flip(SiteNum, tag, i, next)
                rtag, l = findstate(ntag, mb, SiteNum)
                if rtag > 0:
                    nind = mb.index(rtag) 
                    if nind in temp:
                        temp[nind] += Jxy/2 * np.exp(fac*k*l) * np.sqrt(R[ind]/R[nind])
                    else:
                        temp[nind] = Jxy/2 * np.exp(fac*k*l) * np.sqrt(R[ind]/R[nind])
        odTemp = sorted(temp)
        for sf in odTemp:
            col.append (ind)
            row.append (sf)
            data.append (temp[sf])
    return col, row, data, dim

In [4]:
SiteNum = 16
Sz = 0

Jz = 1
Jxy = 1

momentum = list(range(-int(SiteNum/2)+1, int(SiteNum/2)+1))

eng = [] #ground state energy
moment = []

for k in momentum:
    col, row, data, dim = heisenberg_T(SiteNum, Jz, Jxy, Sz, k)
    print('Hilbert space dimension for momentum',k,'is:', dim)

    ham = csc_matrix ((data, (row, col)), shape=(dim, dim))
    vals, vecs = eigsh (ham, k=5, which = 'SA')
    
    moment.append(k) 
    eng.append(vals[0])

gs = min(eng)
gind = eng.index(gs)

print('The ground state energy is:',gs)
print('Corresponding momentum is:', moment[gind],'* 2pi/N')


Hilbert space dimension for momentum -7 is: 800
Hilbert space dimension for momentum -6 is: 808
Hilbert space dimension for momentum -5 is: 800
Hilbert space dimension for momentum -4 is: 809
Hilbert space dimension for momentum -3 is: 800
Hilbert space dimension for momentum -2 is: 808
Hilbert space dimension for momentum -1 is: 800
Hilbert space dimension for momentum 0 is: 810
Hilbert space dimension for momentum 1 is: 800
Hilbert space dimension for momentum 2 is: 808
Hilbert space dimension for momentum 3 is: 800
Hilbert space dimension for momentum 4 is: 809
Hilbert space dimension for momentum 5 is: 800
Hilbert space dimension for momentum 6 is: 808
Hilbert space dimension for momentum 7 is: 800
Hilbert space dimension for momentum 8 is: 810
The ground state energy is: -7.142296360616822
Corresponding momentum is: 0 * 2pi/N
